In [1]:
import astropy.units as u
import astropy.constants as C
import importlib
import units.smm as smu
importlib.reload(smu)

<module 'units.smm' from 'e:\\Code\\autism\\ss13\\supermattercore\\units\\smm.py'>

In [4]:
smu.POWERED_EMITTER_DMG*8

<Quantity 1137.76 hitpts>

In [14]:
(u.MeV / u.s/ u.cm**3)

Unit("MeV / (s cm3)")

In [17]:
u.ST.names

['ST', 'STflux']

In [5]:
(113/700)**3

0.004206696793002915

In [2]:
%config Application.logging_config = {'loggers': {'ct.sm': {'propagate': True, 'level': 'DEBUG', 'handlers': ['console']}}}

In [20]:
canist_ref = smu.GasMixture.pure('N2', 293.15*u.K, 1000*u.L, 90*smu.atm)
canist_ref

GasMixture(T=<Quantity 293.15 K>, V=<Quantity 1000. l>, nus={'N2': <Quantity 3743.41692471 mol>})

In [14]:
he = smu.HeatExchanger(191)
# f'{(canist.T - he.approx_cool(canist)):.5g}'
# he.cool(canist)

In [32]:
loop = smu.GasMixture(0*u.K, ((15.68 + .2*2)*u.m**3).to(u.L))
canist = canist_ref * 1

In [33]:
pump = smu.Pump()
canist.V += pump.V_in
loop.V += pump.V_out    

In [34]:
for _ in range(40):
    pump.pump(canist, loop)
    he.cool(loop)
print(canist.p.to(u.kPa), loop.p.to(u.kPa))
print('Changing canister to new...')
canist = canist_ref * 1
canist.V += pump.V_in
for _ in range(40):
    pump.pump(canist, loop)
    he.cool(loop)
print(canist.p.to(u.kPa), loop.p.to(u.kPa))
print('Changing canister to new...')
canist = canist_ref * 1
canist.V += pump.V_in
for _ in range(40):
    pump.pump(canist, loop)
    he.cool(loop)
print(canist.p.to(u.kPa), loop.p.to(u.kPa))
# %config Application.log_level = 'INFO'
# for _ in range(5):
#     pump.pump(canist, loop)
#     he.cool(loop)
#     # he.cool(loop) # Gives better correlation with actual
# %config Application.log_level = 30

5.170446323506409 kPa 551.0484772953529 kPa
Changing canister to new...
5.27884364306323 kPa 1092.7835406266563 kPa
Changing canister to new...
10.664358047496586 kPa 1625.5100418895684 kPa


In [19]:
loop.p.to(u.kPa), loop.T

(<Quantity 558.28368807 kPa>, <Quantity 289.8045406 K>)

In [433]:
%config Application.log_level = 'INFO'
for _ in range(5):
    pump.pump(canist, loop)
%config Application.log_level = 30

[ct.sm.Pump] GasMixture(T=<Quantity 293.15 K>, V=<Quantity 200. l>, nus={'N2': <Quantity 623.90282079 mol>})
[ct.sm.Pump] Source entropy 155.129 J / (K mol) -> Sink entropy 150000 J / (K mol)
[ct.sm.Pump] Specific entropy change 149845 J / (K mol)
[ct.sm.Pump] Specific power 0
[ct.sm.Pump] Transferred 623.903 mol


[ct.sm.Pump] GasMixture(T=<Quantity 293.15 K>, V=<Quantity 200. l>, nus={'N2': <Quantity 519.91901732 mol>})
[ct.sm.Pump] Source entropy 156.608 J / (K mol) -> Sink entropy 191.439 J / (K mol)
[ct.sm.Pump] Specific entropy change 34.8311 J / (K mol)
[ct.sm.Pump] Specific power 0
[ct.sm.Pump] Transferred 519.919 mol
[ct.sm.Pump] GasMixture(T=<Quantity 293.15 K>, V=<Quantity 200. l>, nus={'N2': <Quantity 433.26584777 mol>})
[ct.sm.Pump] Source entropy 158.094 J / (K mol) -> Sink entropy 186.405 J / (K mol)
[ct.sm.Pump] Specific entropy change 28.3109 J / (K mol)
[ct.sm.Pump] Specific power 0
[ct.sm.Pump] Transferred 433.266 mol
[ct.sm.Pump] GasMixture(T=<Quantity 293.15 K>, V=<Quantity 200. l>, nus={'N2': <Quantity 361.05487314 mol>})
[ct.sm.Pump] Source entropy 159.584 J / (K mol) -> Sink entropy 183.737 J / (K mol)
[ct.sm.Pump] Specific entropy change 24.1532 J / (K mol)
[ct.sm.Pump] Specific power 0
[ct.sm.Pump] Transferred 361.055 mol
[ct.sm.Pump] GasMixture(T=<Quantity 293.15 K>, V=